Imports

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from statannotations.Annotator import Annotator

Load data and group single cell data based on Well & Replicate

In [51]:
# Load the CSV files of replicate experimental data
data_exp03 = pd.read_csv('Data/Data_Exp03_CP_HC_SingleCells.csv')
data_exp05 = pd.read_csv('Data/Data_Exp05_CP_HC_SingleCells.csv')
data_exp06 = pd.read_csv('Data/Data_Exp06_CP_HC_SingleCells.csv')

data_exp03['Replicate'] = 'Exp03'
data_exp05['Replicate'] = 'Exp05'
data_exp06['Replicate'] = 'Exp06'

# Merge the dataframes into a single dataframe
main_df = pd.concat([data_exp03, data_exp05, data_exp06]).reset_index(drop=True)

#Keep single cell data
single_cell_df = main_df.copy()

# Group the data based on 'Replicate' and 'Well' and calculate the mean
main_df = main_df.groupby(['Replicate', 'Category', 'Metadata_Well']).mean().reset_index()

num_rows = len(main_df)
print("Number of rows:", num_rows)

Number of rows: 114


C:\Users\JohannesWilbertz\AppData\Local\Temp\ipykernel_33568\1336435514.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  main_df = main_df.groupby(['Replicate', 'Category', 'Metadata_Well']).mean().reset_index()


Plot settings

In [ ]:
# What to plot?
data = main_df.copy()
x = 'Category'
y = 'Granularity_10_OrigMito' # AreaShape_Area, Intensity_IntegratedIntensity_OrigAGP, Granularity_10_OrigMito
ytitle = y
order = ['Untreated', 'LPS', 'Nigericin', 'LPS/Nigericin']
hue = 'Category'

# Set colors
# Define a dictionary that maps categories to colors
color_dict = {'Untreated': 'gray', 'LPS': 'orange', 'Nigericin': 'red', 'LPS/Nigericin': 'purple'}

# Set the style and font
plt.rcParams["font.family"] = "Arial"
sns.set_context('talk')

Create super plot showing all data points and replicate medians

In [ ]:
# Create the plot
plt.figure(figsize=(3, 4))

# Boxplot
ax = sns.boxplot(x=x, y=y, data=data, showfliers=False, hue=hue, order=order, zorder=1, width=0.5, linewidth=2, dodge=False)  
# Transparancy of boxplot filling color
for patch in ax.patches: #patches
    r, g, b, a = patch.get_facecolor()
    patch.set_facecolor((r, g, b, 0))

ax = sns.stripplot(x=x, y=y, data=data, size=7, alpha=0.2, zorder=0, hue=hue, order=order, jitter=0.2, palette=color_dict, dodge=False)

# Add replicate averages on top of plot
ReplicateAverages = data.groupby([x, 'Replicate'], as_index=False).agg({y: "median"})
ReplicateAverages = ReplicateAverages.sort_values([x, 'Replicate']) # Sorting is necessary to plot averages accurately on dataclouds

ax = sns.stripplot(x=x, y=y, size=10, edgecolor="k", linewidth=2, palette=color_dict, data=ReplicateAverages, order=order, jitter=0.1, hue=hue, dodge=False)

# Figure aesthetics
ax.set(xlabel=None)
ax.set(ylabel=y)
ax.set_ylim(0, None)
sns.despine()
ax.get_legend().remove()
ax.set_xticklabels(ax.get_xticklabels(),rotation=45, ha='right', rotation_mode="anchor")

# Add statistical annotation on top of figure
pairs=[
    ("Untreated", "LPS"),
    ("Untreated", "Nigericin"),
    ("Untreated", "LPS/Nigericin")
    ]

annot = Annotator(ax, pairs, data=ReplicateAverages, x=x, y=y, order=order)
annot.configure(test='t-test_ind', verbose=2, loc='outside')
annot.apply_test()
annot.annotate()

# Saving plot
plt.savefig('CP_'+ y + '.pdf', bbox_inches='tight')

plt.show()